In [ ]:
#| hide
from rlm.core import *

# RLM

> Recursive Language Models for ontology-based query construction

This project implements the Recursive Language Model (RLM) architecture for querying RDF ontologies through progressive disclosure. The implementation follows the protocol from [Zhang et al. (2025)](https://github.com/alexzhang13/rlm) while using [claudette](https://claudette.answer.ai/) as the LLM backend.

The work is part of an ongoing investigation into how LLM agents can navigate large knowledge graphs without overwhelming their context windows. Rather than loading entire ontologies into the prompt, the agent iteratively explores through bounded REPL operations, delegating heavy summarization tasks to sub-LLMs.

## Background

The RLM architecture addresses a fundamental tension in using LLMs for knowledge graph tasks: ontologies and query results are often too large to fit in context, yet the model needs semantic understanding to construct correct queries.

The solution externalizes the large context to a REPL environment. The root LLM emits small code blocks that execute against the graph, receiving truncated results. When more detail is needed, it delegates to sub-LLMs via `llm_query()` calls that summarize specific chunks. The process iterates until the model returns a final answer.

This implementation extends RLM with two complementary memory systems:

**Dataset memory** (RDF quads) stores domain facts discovered during exploration. An RDF Dataset provides named graphs for working memory (`mem`), provenance tracking (`prov`), and scratch space (`work/*`). Facts persist across queries and can be snapshotted for session continuity.

**Procedural memory** ([ReasoningBank](https://arxiv.org/html/2509.25140v1)-style) stores reusable methods extracted from past trajectories. After each RLM run, a judge evaluates success or failure, and an extractor distills procedural insights: query templates, debugging strategies, exploration heuristics. These are retrieved via BM25 for similar future tasks, allowing the agent to improve over time without relearning from scratch.

Additional components include SPARQL result handles that expose metadata without materializing full result sets, SHACL shape indexing for schema discovery, and query template retrieval from `sh:SPARQLExecutable` examples.

## Installation

This project uses [uv](https://github.com/astral-sh/uv) for package management with a shared environment:

```bash
source ~/uvws/.venv/bin/activate
uv pip install fastcore claudette rdflib rank-bm25
uv pip install -e .
```

For development, you also need nbdev:

```bash
uv pip install nbdev
nbdev_install_hooks
```

## Example

The following demonstrates loading an ontology into the dataset memory and using bounded view functions to explore it.

In [ ]:
from rlm.dataset import setup_dataset_context

# Initialize dataset with mem/prov graphs
ns = {}
setup_dataset_context(ns)
print(ns['dataset_stats']())

Dataset 'ds' (session: ae4334a3)
mem: 0 triples
prov: 0 events
work graphs: 0
onto graphs: 0


In [ ]:
# Mount an ontology (SHACL shapes are auto-indexed)
ns['mount_ontology']('ontology/dcat-ap/dcat-ap-SHACL.ttl', 'dcat')

# The SHACL index is now available
print(ns['dcat_shacl'].summary())

SHACLIndex: 42 shapes, 146 keywords, paradigm=validation


In [ ]:
from rlm.shacl_examples import search_shapes, describe_shape

# Search for shapes related to datasets
results = search_shapes(ns['dcat_shacl'], 'dataset', limit=3)
for r in results:
    print(f"{r['uri'].split('#')[-1]}: targets {r['targets']}")

dcat:CatalogShape: targets ['http://www.w3.org/ns/dcat#Catalog']
dcat:DatasetShape: targets ['http://www.w3.org/ns/dcat#Dataset']
dcat:DataServiceShape: targets ['http://www.w3.org/ns/dcat#DataService']


In [ ]:
# Get bounded description of a shape (first 10 properties)
desc = describe_shape(ns['dcat_shacl'], results[0]['uri'], limit=10)
print(f"Properties: {desc['property_count']} (showing {len(desc['properties'])})")
for p in desc['properties'][:5]:
    print(f"  {p['path'].split('/')[-1]}: min={p.get('minCount')}")

Properties: 44 (showing 10)
  spatial: min=None
  hasPart: min=None
  dcat#dataset: min=None
  dcat#themeTaxonomy: min=None
  applicableLegislation: min=None


Query templates can be loaded from SHACL-AF examples and searched by keyword:

In [ ]:
from rlm.shacl_examples import load_query_examples, search_queries, get_query_text

# Load neXtProt SPARQL examples
load_query_examples('ontology/uniprot/examples/neXtProt', ns, 'nxq')
print(ns['nxq'].summary())

# Find queries about phosphorylation
queries = search_queries(ns['nxq'], 'phosphorylation', limit=2)
for q in queries:
    print(f"{q['uri'].split('/')[-1]}: {q['comment'][:60]}...")

QueryIndex: 776 queries, 1507 keywords
NXQ_00216: Phosphorylation sites from PeptideAtlas Phosphoproteome...
NXQ_00001: Proteins phosphorylated and located in the cytoplasm...


## Project Structure

The implementation follows nbdev conventions. Source notebooks are in `nbs/`, with modules exported to `rlm/`.

```
nbs/
├── 00_core.ipynb           # RLM loop, llm_query, FINAL_VAR
├── 01_ontology.ipynb       # Ontology loading and namespace binding
├── 02_dataset_memory.ipynb # RDF Dataset with mem/prov/work graphs
├── 03_sparql_handles.ipynb # SPARQL result handles and bounded views
├── 05_procedural_memory.ipynb # ReasoningBank-style procedural memory
└── 06_shacl_examples.ipynb # SHACL indexing and query templates
```

Design documents and the project trajectory are in `docs/`. See [docs/planning/trajectory.md](docs/planning/trajectory.md) for the implementation roadmap.

## Status

This is preliminary research code under active development. The current implementation covers stages 1-4 of the trajectory:

- Stage 1: Core RLM loop with claudette backend
- Stage 2: Bounded view primitives for progressive disclosure
- Stage 3: SPARQL handles with work-bound query execution
- Stage 4: SHACL shape indexing and query template retrieval

Stages 5-6 (full ontology trajectories and evaluation framework) are planned but not yet implemented.

The code is developed through exploratory programming in Jupyter notebooks using nbdev. It targets integration with the [Solveit](https://solveit.ai) platform but can run standalone.

## References

- Zhang, A., et al. (2025). [Recursive Language Models](https://github.com/alexzhang13/rlm). The reference implementation this project follows.
- Wang, B., et al. (2025). [ReasoningBank: Self-Evolving Procedural Knowledge for Adaptive Reasoning](https://arxiv.org/html/2509.25140v1). Procedural memory approach for learning from trajectories.
- Anthropic. (2025). [Building Effective Agents](https://www.anthropic.com/research/building-effective-agents). Context engineering patterns for agentic systems.
- Howard, J. & Gugger, S. [nbdev](https://nbdev.fast.ai/). Literate programming framework.
- Howard, J. [claudette](https://claudette.answer.ai/). Claude API wrapper used as the LLM backend.